# Train a Reasoning LLM with GRPO (R1-Zero Replication)

This notebook trains **Qwen2.5-3B** using pure RL (GRPO) to develop reasoning capabilities,
replicating the DeepSeek-R1-Zero experiment.

**Primary target:** H100 80GB (~6h, ~$18 on Jarvis Labs)
**Also supports:** A100 80GB (falls back to 1.5B config), A100 40GB, and smaller GPUs.

## 1. Setup

In [ ]:
# Install dependencies
!pip install -q torch transformers trl accelerate datasets math-verify wandb tensorboard pyyaml

In [ ]:
# Clone the repo (or upload files)
import os
REPO_URL = "https://github.com/manojkgorle/smol-reason"
PROJECT_DIR = "/content/smol-reason"

if REPO_URL and not os.path.exists(PROJECT_DIR):
    !git clone {REPO_URL} {PROJECT_DIR}
elif not os.path.exists(PROJECT_DIR):
    print("Please upload the reson-llm project files or set REPO_URL")

os.chdir(PROJECT_DIR)
print(f"Working directory: {os.getcwd()}")

In [ ]:
# GPU check and memory-based config selection
import torch

if not torch.cuda.is_available():
    raise RuntimeError("No GPU detected! This notebook requires a GPU.")

gpu_name = torch.cuda.get_device_name(0)
gpu_mem_gb = torch.cuda.get_device_properties(0).total_mem / 1e9
print(f"GPU: {gpu_name}")
print(f"Memory: {gpu_mem_gb:.1f} GB")

# Auto-select config based on GPU type and memory
# H100 80GB → 3B model (best results, full utilization of H100)
# A100 80GB → 1.5B model with G=16 (full config)
# A100 40GB → 1.5B model with G=8 (reduced)
# Other     → 1.5B model with minimal config
is_h100 = "H100" in gpu_name or "H200" in gpu_name

if is_h100 and gpu_mem_gb >= 70:
    GPU_TIER = "h100_80gb"
    CONFIG_FILE = "configs/grpo_qwen2.5_3b_h100.yaml"
    print("Config: Qwen2.5-3B on H100 (G=12, max_completion=1536)")
elif gpu_mem_gb >= 70:
    GPU_TIER = "a100_80gb"
    CONFIG_FILE = "configs/grpo_qwen2.5_1.5b.yaml"
    print("Config: Qwen2.5-1.5B full (G=16, max_completion=2048)")
elif gpu_mem_gb >= 35:
    GPU_TIER = "40gb"
    CONFIG_FILE = "configs/grpo_qwen2.5_1.5b.yaml"
    print("Config: Qwen2.5-1.5B reduced (G=8, max_completion=1024)")
else:
    GPU_TIER = "small"
    CONFIG_FILE = "configs/grpo_qwen2.5_1.5b.yaml"
    print("WARNING: Limited GPU memory. Using minimal config.")

In [ ]:
# W&B login
import wandb
wandb.login()

In [ ]:
# Google Drive mount for checkpoint persistence
from google.colab import drive
drive.mount("/content/drive")

DRIVE_CHECKPOINT_DIR = "/content/drive/MyDrive/smol-reason-checkpoints"
os.makedirs(DRIVE_CHECKPOINT_DIR, exist_ok=True)
print(f"Checkpoints will be saved to: {DRIVE_CHECKPOINT_DIR}")

## 2. Configure Training

In [ ]:
import yaml

# Load the auto-selected config
print(f"Loading config: {CONFIG_FILE}")
with open(CONFIG_FILE) as f:
    config = yaml.safe_load(f)

# Override output_dir to use Google Drive
config["output_dir"] = DRIVE_CHECKPOINT_DIR

# Apply GPU-tier-specific overrides for non-H100/non-80GB GPUs
if GPU_TIER == "40gb":
    config["num_generations"] = 8
    config["max_completion_length"] = 1024
    config["per_device_train_batch_size"] = 1
    print("Adjusted config for 40GB GPU")
elif GPU_TIER == "small":
    config["num_generations"] = 4
    config["max_completion_length"] = 512
    config["per_device_train_batch_size"] = 1
    config["gradient_accumulation_steps"] = 4
    print("Adjusted config for small GPU")

# Write adjusted config
RUNTIME_CONFIG = "/content/runtime_config.yaml"
with open(RUNTIME_CONFIG, "w") as f:
    yaml.dump(config, f, default_flow_style=False)

print(f"\nModel: {config['model_name_or_path']}")
print(f"GPU tier: {GPU_TIER}")
print(f"\nTraining config:")
for k, v in config.items():
    print(f"  {k}: {v}")

## 3. Train

In [ ]:
# Check for existing checkpoint to resume from
import glob

checkpoints = sorted(glob.glob(f"{DRIVE_CHECKPOINT_DIR}/checkpoint-*"))
resume_arg = ""
if checkpoints:
    latest = checkpoints[-1]
    print(f"Found checkpoint: {latest}")
    resume_arg = f"--resume_from_checkpoint {latest}"
else:
    print("No checkpoint found, starting fresh.")

In [ ]:
# Launch training
!python src/train_grpo.py --config {RUNTIME_CONFIG} {resume_arg}

## 4. Monitor (TensorBoard)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir {DRIVE_CHECKPOINT_DIR}

## 5. Evaluate

In [ ]:
# Quick eval on 200 samples
!python src/evaluate.py \
    --model_path {DRIVE_CHECKPOINT_DIR} \
    --num_samples 200 \
    --output_dir eval_results

print(f"\nModel evaluated: {config['model_name_or_path']} (trained)")

In [ ]:
# Baseline comparison (raw base model, before RL training)
BASE_MODEL = config["model_name_or_path"]
print(f"Evaluating baseline: {BASE_MODEL}")
!python src/evaluate.py \
    --model_path {BASE_MODEL} \
    --num_samples 200 \
    --output_dir eval_results_baseline

In [ ]:
# Compare results
import json

with open("eval_results/summary.json") as f:
    trained = json.load(f)
with open("eval_results_baseline/summary.json") as f:
    baseline = json.load(f)

print("=" * 60)
print(f"{'Metric':<30} {'Baseline':>12} {'Trained':>12}")
print("=" * 60)
for dataset in ["gsm8k", "math"]:
    if dataset in trained and dataset in baseline:
        b = baseline[dataset]
        t = trained[dataset]
        print(f"{dataset.upper()} accuracy:{'':>13} {b['accuracy']:>11.1%} {t['accuracy']:>11.1%}")
        print(f"{dataset.upper()} format compliance:{'':>4} {b['format_compliance']:>11.1%} {t['format_compliance']:>11.1%}")
        print(f"{dataset.upper()} avg think tokens:{'':>5} {b['avg_think_tokens']:>11.0f} {t['avg_think_tokens']:>11.0f}")
        print("-" * 60)

## 6. Push to Hub (Optional)

In [ ]:
# Uncomment and set your HF username to push
# from huggingface_hub import login
# login()

# HF_USERNAME = "your-username"
# MODEL_TAG = config["model_name_or_path"].split("/")[-1].lower()
# REPO_NAME = f"{HF_USERNAME}/{MODEL_TAG}-r1zero-grpo"
# from transformers import AutoModelForCausalLM, AutoTokenizer
# model = AutoModelForCausalLM.from_pretrained(DRIVE_CHECKPOINT_DIR, torch_dtype=torch.bfloat16)
# tokenizer = AutoTokenizer.from_pretrained(DRIVE_CHECKPOINT_DIR)
# model.push_to_hub(REPO_NAME)
# tokenizer.push_to_hub(REPO_NAME)
# print(f"Pushed to: https://huggingface.co/{REPO_NAME}")